### Initial Setup

The following cells install dependencies and set up LLM tracing.


In [1]:
# load required env vars for ddtrace
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import ddtrace.auto

ddtrace.patch_all()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

In [3]:
from openai import OpenAI
import os

oai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

sys_prompt = """
Your task is to 
1. Summarize the given text at a 6th grade reading level in no more than 2 sentences.
2. Identify what topics the text belongs to that would allow you to categorize it in a school library.
Format your output strictly following this JSON convention:
{	
    "topics": <[insert array of topics here]>
    "summary": <insert summary here>
}
	"""


def summarize(text, prompt=sys_prompt):
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": text},
    ]
    # llm span auto-instrumented via our openai integration
    return (
        oai_client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo",
            response_format={"type": "json_object"},
        )
        .choices[0]
        .message.content
    )